In [ ]:
from IPython.core.display import HTML
def css_styling():
    styles = open("./custom.css", "r").read()
    return HTML(styles)
css_styling()

In [ ]:
import logging
logging.getLogger("bokeh.embed.util").setLevel(logging.ERROR)

In [ ]:
import acoular as ac
import numpy as np
import matplotlib.pyplot as plt
import os

<img src="img/DAGA_logo.png" alt="DAGA Logo" style="width:160px; position:absolute; top:20px; right:0px;">
<img src="img/TU-lang.png" alt="TU Logo" style="width:230px; position:absolute; top:0px; right:180px;">

<h1 style="margin-top: 180px; margin-bottom: 50px; color: #A81D1E;">Acoustic Data Acquisition and Processing in Python using a MEMS Microphone Array</h1>
<h3 style="margin-top: 50px; margin-bottom: 50px; color: #434343;">Rabea Eschenhagen, Adam Kujawski, Art J. R. Pelling, Mikolaj Czuchaj, Gert Herold, Oliver Lylloff, Ennes Sarradj</h3>

<h5 style="margin-top: 50px; margin-bottom: 100px; color: #434343;">Jupyter Notebook available at <a href="https://github.com/acoular/workshop_DAS-DAGA-2025">https://github.com/acoular/workshop_DAS-DAGA-2025</a></h5>


## What we will do

- Stream and process live audio data from the laptop's microphone
- Revisit a simple beamforming example using microphone array data
- Demonstrate an application that integrates the discussed concepts: Acoular extension _SpectAcoular_

## Getting data from input channels

### Sounddevice library
- Current environment must include the `sounddevice` library
- First step: Check the available audio devices on the system

In [ ]:
import sounddevice as sd

sd.query_devices() 

### Set index of the desired audio device

In [ ]:
device_number = None #TODO: Set the device number here

###  Acoular's SoundDeviceSamplesGenerator

- `SoundDeviceSamplesGenerator` class handles audio data streams
- Uses the sounddevice library internally
- Requires setting the device index to choose the audio interface
- Configuration for single-channel acquisition (numchannels=1)

In [ ]:
dev = ac.SoundDeviceSamplesGenerator(
    device = device_number,
    numchannels = 1,
)

print("Sampling frequency: ", dev.sample_freq)

### Transform voltages into sound pressure 

- Recorded signal is in volts and needs conversion to sound pressure (Pa)
- Sensitivity defines voltage output per unit of sound pressure
- Conversion is done by multiplying with the reciprocal of sensitivity
- _Example: 0.016 V/Pa means 1 Pa corresponds to 0.016 V_


In [ ]:
dev_calibrated = ac.Calib(source=dev, data=1/0.016*np.ones(dev.num_channels))

### Record a signal

- Acquire a signal with a duration of 2 seconds
- Read input stream block by block (block size: 256 samples)
- Use `return_result` to retrieve all blocks at once
- `SoundDeviceSamplesGenerator` prints device properties when streaming starts
- Array `t`: The first axis is time data, and the second axis is the number of channels

In [ ]:
recording_time = 2 # seconds 
dev.num_samples = int(recording_time * dev.sample_freq)

signal = ac.tools.return_result(dev_calibrated, num=256)
t = np.arange(signal.shape[0]) / dev.sample_freq 

### Plot of results


In [ ]:
plt.figure(figsize=(15,4))
plt.plot(t, signal[:,0])
plt.ylabel('Sound Pressure [Pa]')
plt.xlabel('Time [s]')
plt.tight_layout()
plt.show()

## Writing measurement data to disk

### Further Processing

- Acoular allows additional processing steps beyond acquiring time data
- Processing is handled via classes from the `tprocess.py` module
- `WriteH5` class saves the stream to an HDF5 file
- Alternatively, `WriteWav` can be used to save the stream as a WAV file

In [ ]:
hdf5 = ac.WriteH5(
    source = dev_calibrated,
    file = 'sounddevice.h5',
)

### Save to disk
- Two processing chains were created to read data from `dev` (`SoundeDeviceSamplesGenerator`)
- Data can either be passed on or saved to disk
- Due to lazy evaluation, no processing has occurred yet
- Two ways to start acquisition and writing:

    1. `hdf5.result(num=256)`: Generates data block-wise
    2. `hdf5.save()`: Runs the processing chain in one go without returning data
- Option 2 is used

In [ ]:
hdf5.save()

### Read file

- File can be processed with any software supporting this format
- Use the `TimeSamples` object to read the data for further post-processing

In [ ]:
ts = ac.TimeSamples(file='sounddevice.h5')

print("Number of channels: ", ts.num_channels, "; Number of samples: ", ts.num_samples)

## Microphone array data processing

### Set up input device parameters
- Set up input device parameters for recording.
- A 16-channel low-cost microphone array was used for recording
- `SoundDeviceSamplesGenerator` initialized with:
    - `device = 0`
    - `numchannels = 16`
- Recording duration: 4 seconds.
- Multichannel data saved in `img/multichannel_signal.h5`

In [ ]:
time_data = ac.MaskedTimeSamples(file='img/multichannel_signal.h5')

print("Number of channels: ", time_data.num_channels, "; Number of samples: ", time_data.num_samples)

### Define the microphone geometry and the computational grid
-   Microphone array geometry is loaded from an XML file

In [ ]:
mics = ac.MicGeom(file=ac.__path__[0] + '/xml/minidsp_uma-16_mirrored.xml')

# Plot of microphone positions
plt.scatter(*mics.pos[:2])
plt.show()

### Define a rectangular grid 
- Specifies the search region for sound sources
- Covers the spatial range with a resolution of 0.05 m

In [ ]:
x_min, x_max = -1.2, 1.2
y_min, y_max = -1.2, 1.2
z = 1.54
increment = 0.05

grid = ac.RectGrid(                 # for 3D Grid use ac.RectGrid3D and set z_min and z_max
    x_min=x_min, x_max=x_max, 
    y_min=y_min, y_max=y_max, 
    z=z, 
    increment=increment
)

### Define the steering vector for beamforming

- Set the speed of sound in air
- Create a steering vector using microphone geometry and a defined rectangular grid

In [ ]:
c = 346 # speed of sound
steer = ac.SteeringVector(mics=mics, env=ac.Environment(c=c), grid=grid)

### Setup beamforming
- Transform the recorded time-domain signal into the frequency-domain
- Apply Hanning window with 50% overlap
- Use a block size of 128 samples
- `BeamformerBase` performs beamforming using the delay-and-sum algorithm

In [ ]:
freq_data = ac.PowerSpectra(source=time_data, window='Hanning', overlap='50%', block_size=128)
bb = ac.BeamformerBase(freq_data=freq_data, steer=steer)

### Apply beamforming 
- _Example_: Use a third-octave frequency band centred at 4000 Hz
- Convert raw beamforming output to sound pressure levels (SPL) in decibels

In [ ]:
frequency = 4000 
result = bb.synthetic(frequency, num=3)
Lm = ac.L_p(result)
Lm_reshaped = Lm.reshape(grid.shape) # converting 1D array to 2D array

### Plot of beamforming results 
- Overlay results onto a background image
- Heatmap represents sound pressure levels (SPL) in decibels
- SPL values are mapped to image coordinates

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

# Background image 
background_img = plt.imread("img/multichannel_signal.jpg") 
im_w, im_h = background_img.shape[1], background_img.shape[0] 
ax.imshow(background_img, extent=[0, im_w, 0, im_h])

# Pixel indices of Beamforming plot in photo for correct placing
p_ind = lambda img, v: (img / 2 + 230 * (v / z)) 

# Beamforming plot
heatmap = ax.imshow(
    Lm_reshaped, extent=[p_ind(im_w, x_min), p_ind(im_w, x_max), p_ind(im_h, y_min), p_ind(im_h, y_max)], 
    origin='lower', cmap='jet', alpha=0.4, vmin=70, vmax=80
)

cbar = fig.colorbar(heatmap, ax=ax)
cbar.set_label("Sound Pressure Level (dB)")

ax.set_xlim([0, im_w])
ax.set_ylim([0, im_h])

ax.axis('off')
plt.show()

## SpectAcoular! Extension for Acoular
### What it is
- Easy and interactive microphone array data processing
- Designed for microphone array data acquisition, analysis, and exploration
- Provides interactive tools for real-time beamforming and signal processing
- Implements graphical controls (widgets) for user interaction
- Customizable and expandable for different applications

### Where to find it
- **GitHub:** [SpectAcoular repository](https://github.com/acoular/spectacoular)
- **Anaconda:** [SpectAcoular package](https://anaconda.org/acoular/spectacoular)
- **Documentation:** [SpectAcoular docs](https://acoular.github.io/spectacoular)
  
### Installation
- With Conda: `conda install acoular::spectacoular`

### How it works
- Uses [_Bokeh_](https://bokeh.org/) for web-based visualization, allowing client-server applications
- Translates class attributes from Acoular into interactive control widgets
- Allows real-time parameter adjustments directly from the user interface
- _Example:_ Creating an Interactive Grid, as shown below




In [ ]:
# Ensure that Bokeh and SpectAcoular are installed in the current environment.
# If these packages are not available, you may skip this cell.

from spectacoular import RectGrid
from bokeh.io import show, output_notebook
from bokeh.layouts import column

grid = RectGrid()
widgets = grid.get_widgets()
output_notebook()
widget_list = list(widgets.values())
col = column(widget_list, sizing_mode="scale_both", width=200)

show(col)

### Pre-build interactive applications

- Module provides various predefined examples and applications
  - to explore the point-spread function of a microphone arrays
  - to evaluate rotating array data
  - to measure sound sources
  - ...
- As an example, we will take a look at the [Microphone array measurement app](https://acoular.github.io/spectacoular/apps/measurementapp.html)


In [ ]:
os.popen('msm_app --args --device=sounddevice')

## Acoular Future Directions Survey – Don't forget to participate! 

<img src="img/survey.png" alt="Survey" style="height: auto; width: 50%;">